This part explores the different approaches for evaluation the WHERE condition

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from Main.combined_pipeline import combined_pipeline
from Utilities.database import query_database
from Evaluation.test_evaluation import evaluate_results

/home/vlapan/Documents/Masterarbeit/Relational/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_and_populate_table_two(dataframe, table_name):
    try:
        # Clean table name (optional)
        table_name = table_name.replace(" ", "_")

        # Drop table if it exists
        delete_table_query = f"DROP TABLE IF EXISTS {table_name} CASCADE;"
        query_database(delete_table_query, printing=False)

        # Convert all data to string format (since we're dealing with text data)
        dataframe = dataframe.astype(str)

        # Create table with two columns (for the two columns in the dataframe)
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (category TEXT, items TEXT);"
        query_database(create_table_query, printing=False)
        print(f"Table '{table_name}' created successfully.")

        # Insert data into table
        values = []
        for _, row in dataframe.iterrows():
            col1_value = row.iloc[0].replace("'", "''")  # Escape single quotes for SQL
            col2_value = row.iloc[1].replace("'", "''")  # Escape single quotes for SQL
            values.append(f"('{col1_value}', '{col2_value}')")  # Insert each row as a pair of column values

        if values:
            # Build the insert query
            insert_query = f"INSERT INTO {table_name} (category, items) VALUES {', '.join(values)};"
            query_database(insert_query, printing=False)
            print(f"Table '{table_name}' populated successfully.")
        else:
            print("No data to insert.")

    except Exception as e:
        print(f"An error occurred: {e}")

In [4]:
# Load your DataFrame
#item_df = pd.read_csv('WHERE_Evaluation/jio_smart_1000.csv')
item_df = pd.read_csv('Data/jio_mart_items.csv')
#item_df=item_df.iloc[0:20,:]
item_df

,category,sub_category,href,items,price
0,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Fresh Dates (Pack) (Approx 450 g - 500 g),109.0
1,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Tender Coconut Cling Wrapped (1 pc) (Approx 90...,49.0
2,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Mosambi 1 kg,69.0
3,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Orange Imported 1 kg,125.0
4,Groceries,Fruits & Vegetables,https://www.jiomart.com/c/groceries/fruits-veg...,Banana Robusta 6 pcs (Box) (Approx 800 g - 110...,44.0
...,...,...,...,...,...
162308,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 5.6 gm Ganesha Silver ...,1669.0
162309,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 4.58 gm Ganesha Silver...,1391.0
162310,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 2.31 gm Ganesha Silver...,834.0
162311,Jewellery,Fine Jewellery,https://www.jiomart.com/c/jewellery/fine-jewel...,Reliance Jewels Ag 99.9 7.76 gm Ganesha Silver...,2276.0


In [40]:
create_and_populate_table_two(item_df.loc[:, ['category', 'items']], "jio_smart")

Table 'jio_smart' created successfully.
Table 'jio_smart' populated successfully.


In [41]:
def create_and_populate_table(dataframe, table_name):
    try:
        # Clean table name (optional)
        table_name = table_name.replace(" ", "_")

        # Drop table if it exists
        delete_table_query = f"DROP TABLE IF EXISTS {table_name} CASCADE;"
        query_database(delete_table_query, printing=False)

        # Convert all data to string format
        dataframe = dataframe.astype(str)

        # Create table with a single column
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (Aggregate TEXT);"
        query_database(create_table_query, printing=False)
        print(f"Table '{table_name}' created successfully.")

        # Insert data into table
        values = []
        for _, row in dataframe.iterrows():
            row_values = "\n".join(row.astype(str))  # Concatenate row values with newline separator
            values.append(f"('{row_values.replace("'", "''")}')")  # Escape single quotes for SQL

        if values:
            insert_query = f"INSERT INTO {table_name} VALUES {', '.join(values)};"
            query_database(insert_query, printing=False)
            print(f"Table '{table_name}' populated successfully.")
        else:
            print("No data to insert.")

    except Exception as e:
        print(f"An error occurred: {e}")

In [42]:


# # Define the categories to exclude
# exclude_categories = ['Home & Kitchen', 'Groceries']

# item_df


# # Filter the DataFrame
# item_df_filtered = item_df[~item_df['category'].isin(exclude_categories)]

# # Display the unique categories in the filtered DataFrame
# print(item_df_filtered['category'].unique())

# index=np.random.randint(0,high=len(item_df_filtered),size=1)[0]
# index

# item_df_filtered.iloc[index:index+2:]

# item_df_filtered.shape

# item_df_filtered=item_df_filtered.reset_index(drop=True)

# # Ensure the dataframe has a fresh index
# item_df_filtered = item_df_filtered.reset_index(drop=True)

# # Set the number of rows to select and the number of categories
# rows_to_select = 1000
# categories = item_df_filtered['category'].unique()

# # Calculate the max rows to sample per category
# samples_per_category = rows_to_select // len(categories)

# # List to store sampled rows
# sampled_rows = []

# # Sample rows from each category
# for category in categories:
#     category_rows = item_df_filtered[item_df_filtered['category'] == category]
    
#     # Adjust sample size to avoid over-sampling
#     sample_size = min(samples_per_category, len(category_rows))
    
#     sampled_category_rows = category_rows.sample(n=sample_size, replace=False)
#     sampled_rows.append(sampled_category_rows)

# # Combine the sampled rows into a new dataframe
# random_rows = pd.concat(sampled_rows)

# # Shuffle the rows to ensure randomness
# random_rows = random_rows.sample(frac=1).reset_index(drop=True)

# # Write the selected rows to a CSV file
# random_rows.to_csv('jio_smart_1000.csv', index=False)


In [43]:
unique_categories = item_df['category'].unique()
current_category = unique_categories[0]
unique_categories 

array(['Beauty', 'Electronics'], dtype=object)

In [44]:
ground_truth= query_database(f"SELECT * FROM jio_smart WHERE category = '{current_category}'",printing=False)

# # Extract the numbers 1, 7, 10 from the tuples
# extracted_numbers = [int(item[0].split(',')[1][:-1]) for item in ground_truth]

# print(extracted_numbers)


In [45]:
ground_truth

[('Beauty', 'Flormar Quick Dry Nail Enamel QD20 Rose Taboo 11 ml')]

In [46]:
# query = f"SELECT * FROM jio_smart WHERE items = '{current_category}'"
# answer,metadata = combined_pipeline(query, aux=True)

In [47]:
prompt=f"SELECT items, category FROM jio_smart JOIN wherejio_smartitemsbeauty_comparison_beauty_table ON jio_smart.items = wherejio_smartitemsbeauty_comparison_beauty_table.synonym"
answer=query_database(prompt,printing=False)
print(answer)

[('Flormar Quick Dry Nail Enamel QD20 Rose Taboo 11 ml', 'Beauty')]


In [48]:
answer

[('Flormar Quick Dry Nail Enamel QD20 Rose Taboo 11 ml', 'Beauty')]

In [55]:
accuracy, precision, recall, f1_score = evaluate_results(ground_truth, answer)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [61]:
def evaluate_results_metric(expected, actual):
    """
    Evaluates classification results and calculates True Positives (TP),
    False Positives (FP), False Negatives (FN), and True Negatives (TN).

    Parameters:
    - expected (set of tuples): Ground truth categories.
    - actual (set of tuples): Predicted categories.

    Returns:
    - dict: A dictionary containing TP, FP, FN, TN counts.
    """
    
    # Handle None cases
    if expected is None:
        expected = set()
    if actual is None:
        actual = set()
    
    # Ensure the inputs are sets
    if type(expected) != set:
        expected = set(expected)
    if type(actual) != set:
        actual = set(actual)

    # Convert tuples to frozensets for comparison
    expected_frozensets = {frozenset(t) for t in expected}
    actual_frozensets = {frozenset(t) for t in actual}

    # Compute True Positives (TP) - correctly classified
    tp = sum(any(actual >= expected for actual in actual_frozensets) for expected in expected_frozensets)

    # Compute False Positives (FP) - incorrect extra classifications
    fp = len([actual for actual in actual_frozensets if not any(actual >= expected for expected in expected_frozensets)])

    # Compute False Negatives (FN) - missing expected classifications
    fn = len(expected_frozensets - {expected for expected in expected_frozensets if any(actual >= expected for actual in actual_frozensets)})


    return tp, fp, fn

In [64]:
TP, FP,FN = evaluate_results_metric(ground_truth, answer)
print(f"True Positive: {TP}")
print(f"False Positives: {FP}")
print(f"False Negatives: {FN}")

True Positive: 1
False Positives: 0
False Negatives: 0
